# Data Cleaning and Feature Engineering

### 1. Setup and Data Import

In [65]:
import pandas as pd
import numpy as np

In [66]:
cal = pd.read_csv('data/raw/calendar2024.csv')
lis = pd.read_csv('data/raw/listings2024.csv') 
rev = pd.read_csv('data/raw/reviews2024.csv')

### 2. Initial Column Cleanup

In [67]:
# Remove empty columns
null_cols_lis = lis.columns[lis.isna().all()].tolist()
lis = lis.drop(columns=null_cols_lis)

# Drop unnecessary columns
lis.drop(columns=['scrape_id', 'host_name', 'picture_url', 'host_url', 'host_thumbnail_url', 'host_picture_url'], inplace=True)
cal.drop(columns=['adjusted_price'], inplace=True)
rev.drop(columns=['reviewer_name'], inplace=True)

### 3. Data Type Conversions

In [68]:
# Helper functions
def convert_to_boolean(df, columns, true_value='t'):
    """Convert specified columns from string indicators to boolean"""
    for col in columns:
        df[col] = df[col] == true_value
    return df

def convert_to_datetime(df, columns):
    """Convert specified columns to datetime"""
    for col in columns:
        df[col] = pd.to_datetime(df[col])
    return df

def convert_to_type(df, columns, dtype):
    """Convert specified columns to given dtype"""
    for col in columns:
        df[col] = df[col].astype(dtype)
    return df

### 4. Apply Type Conversions

In [69]:
# Boolean conversions
boolean_cols = ['instant_bookable', 'host_is_superhost', 'host_has_profile_pic', 'host_identity_verified', 'has_availability']
lis = convert_to_boolean(lis, boolean_cols)
cal['available'] = cal['available'] == 't'

# Datetime conversions
datetime_cols_lis = ['calendar_last_scraped', 'first_review', 'last_review', 'last_scraped', 'host_since']
lis = convert_to_datetime(lis, datetime_cols_lis)
cal['date'] = pd.to_datetime(cal['date'])
rev['date'] = pd.to_datetime(rev['date'])

# String conversions
string_columns = ['bathrooms_text', 'neighbourhood', 'neighbourhood_cleansed', 'property_type', 'room_type', 
                 'host_location', 'host_about', 'host_neighbourhood', 'listing_url', 'host_response_time', 
                 'source', 'name', 'description', 'neighborhood_overview']
lis = convert_to_type(lis, string_columns, "string")
rev['comments'] = rev['comments'].astype("string")

### 5. Handle Percentage and Currency Columns

In [70]:
# Convert percentage columns
percentage_cols = ['host_response_rate', 'host_acceptance_rate']
for col in percentage_cols:
    lis = lis.rename(columns={col: f"{col}_pct"})
    lis[f"{col}_pct"] = lis[f"{col}_pct"].str.rstrip('%').astype('float') / 100

# Clean and standardize price columns
lis['price'] = lis['price'].str.replace(r'[\$,]', '', regex=True)
lis = lis.rename(columns={'price': 'price_DKK'})
lis['price_DKK'] = pd.to_numeric(lis['price_DKK'], errors='coerce')

cal['price'] = cal['price'].str.replace(r'[\$,]', '', regex=True)
cal = cal.rename(columns={'price': 'price_USD'})
cal['price_USD'] = pd.to_numeric(cal['price_USD'], errors='coerce')

### 6. Process List-Type Columns

In [71]:
# Process amenities
lis['amenities_count'] = lis.amenities.str.strip('[]').str.split(',').str.len()

# Process host verifications
lis['host_verifications'] = lis['host_verifications'].str.strip('[]').str.replace("'", "").str.split(', ')
verification_dummies = lis['host_verifications'].str.join('|').str.get_dummies()
verification_dummies = verification_dummies.add_prefix('verification_')
lis = pd.concat([lis, verification_dummies], axis=1)

### 7. Handle Missing Values

#### Imputation for Listings

In [72]:
# Physical characteristics imputation - revised approach
# Bathrooms - keep room_type median as it must exist
lis['bathrooms'] = lis.groupby('room_type')['bathrooms'].transform(lambda x: x.fillna(x.median()))
lis['bathrooms'] = lis['bathrooms'].fillna(1.0)  # Fallback to 1 if still missing

# Bedrooms - set to 0 for shared/studio, impute for homes
lis.loc[lis['room_type'].isin(['Shared room', 'Private room']), 'bedrooms'] = \
    lis.loc[lis['room_type'].isin(['Shared room', 'Private room']), 'bedrooms'].fillna(0)
lis.loc[lis['room_type'] == 'Entire home/apt', 'bedrooms'] = \
    lis.loc[lis['room_type'] == 'Entire home/apt', 'bedrooms'].transform(lambda x: x.fillna(x.median()))
lis['bedrooms'] = lis['bedrooms'].fillna(0)  # Any remaining missing to 0

# Beds - ensure at least 1 bed per listing based on accommodates
lis['beds'] = lis['beds'].fillna(lis['accommodates'].clip(lower=1))


# Price imputation
# If price is missing, use the median price for that room type
lis['price_DKK'] = lis.groupby('room_type')['price_DKK'].transform(lambda x: x.fillna(x.median()))

# If any prices are still missing (very rare case), use overall median
lis['price_DKK'] = lis['price_DKK'].fillna(lis['price_DKK'].median())



# Numeric imputation - ensure all numeric fields have appropriate values for PostgreSQL numeric types
numeric_cols_to_impute = {
    'host_response_rate_pct': 0.0,  # Use 0 instead of mean for percentage
    'host_acceptance_rate_pct': 0.0,  # Use 0 instead of mean for percentage
    'review_scores_rating': 0,  # Use 0 instead of median for scores
    'review_scores_accuracy': 0,
    'review_scores_cleanliness': 0,
    'review_scores_checkin': 0,
    'review_scores_communication': 0,
    'review_scores_location': 0,
    'review_scores_value': 0,
    'reviews_per_month': 0,
    'host_listings_count': 1,
    'host_total_listings_count': 1
}

for col, value in numeric_cols_to_impute.items():
    lis[col] = lis[col].fillna(value)

# Categorical imputation - use empty string for VARCHAR fields where appropriate
categorical_cols_to_impute = {
    'host_response_time': '',  # VARCHAR
    'host_neighbourhood': '',  # VARCHAR
    'bathrooms_text': '',  # VARCHAR
    'neighbourhood': 'mode',  # Keep mode for geographic consistency
    'neighbourhood_cleansed': 'mode',  # Keep mode for geographic consistency
    'host_location': 'Copenhagen, Denmark',  # Important location default
    'host_verifications': '[]'  # JSON-compatible empty array
}

for col, strategy in categorical_cols_to_impute.items():
    if strategy == 'mode':
        lis[col] = lis[col].fillna(lis[col].mode()[0])
    else:
        lis[col] = lis[col].fillna(strategy)

# Text columns - use empty string for TEXT fields
text_cols_to_impute = ['host_about', 'neighborhood_overview', 'description']
for col in text_cols_to_impute:
    lis[col] = lis[col].fillna('')

# Date columns - use explicit PostgreSQL-compatible dates
current_date = pd.Timestamp.now().date()
lis['host_since'] = lis['host_since'].fillna(pd.Timestamp('2000-01-01'))  # Use explicit default date
lis['first_review'] = lis['first_review'].fillna(pd.NaT)  # Keep as NULL for no reviews
lis['last_review'] = lis['last_review'].fillna(pd.NaT)  # Keep as NULL for no reviews

# Boolean columns - ensure True/False (not NULL)
boolean_cols = ['instant_bookable', 'host_is_superhost', 'host_has_profile_pic', 
                'host_identity_verified', 'has_availability']
for col in boolean_cols:
    lis[col] = lis[col].fillna(False)  # Default to False for missing booleans

#### Imputation for Calendar

In [73]:
# Price imputation for calendar
cal['price_USD'] = cal.groupby(['listing_id', 'available'])['price_USD'].transform(lambda x: x.fillna(x.median()))
cal['price_USD'] = cal['price_USD'].fillna(0)  # Default to 0 for any remaining NULL prices

# Handle minimum and maximum nights
cal['minimum_nights'] = cal.groupby('listing_id')['minimum_nights'].transform(lambda x: x.fillna(x.median()))
cal['maximum_nights'] = cal.groupby('listing_id')['maximum_nights'].transform(lambda x: x.fillna(x.median()))
# Default to 1 and 365 if still missing
cal['minimum_nights'] = cal['minimum_nights'].fillna(1)
cal['maximum_nights'] = cal['maximum_nights'].fillna(365)

#### Imputation for Reviews

In [74]:
# Handle missing comments
rev['comments'] = rev['comments'].fillna('')  # Empty string for TEXT field

# Handle missing dates (shouldn't be any, but just in case)
rev['date'] = rev['date'].fillna(pd.Timestamp('2000-01-01'))  # Use explicit default date

In [75]:
# After all imputations, check which columns still have nulls
null_check = lis.isnull().sum()
print("\nColumns with remaining NULL values:")
print(null_check[null_check > 0])

# Fix remaining nulls based on their data types
for col in lis.columns[lis.isnull().any()]:
    dtype = lis[col].dtype
    
    if np.issubdtype(dtype, np.number):  # Numeric columns
        lis[col] = lis[col].fillna(0)
    elif dtype == 'datetime64[ns]':  # DateTime columns
        lis[col] = lis[col].fillna(pd.Timestamp('2000-01-01'))
    elif dtype == 'bool':  # Boolean columns
        lis[col] = lis[col].fillna(False)
    else:  # String/object columns
        lis[col] = lis[col].fillna('')


Columns with remaining NULL values:
first_review    3220
last_review     3220
dtype: int64


In [76]:
# Verify imputation
print("\nVerifying no NULL values remain:")
print("\nListings nulls:", lis.isnull().sum().sum())
print("Calendar nulls:", cal.isnull().sum().sum())
print("Reviews nulls:", rev.isnull().sum().sum())


Verifying no NULL values remain:

Listings nulls: 0
Calendar nulls: 0
Reviews nulls: 0


### 8. Feature Engineering

In [77]:
# Store original columns
original_columns = lis.columns.copy()

In [78]:
# Listing Quality Indicators
lis['is_superhost'] = lis['host_is_superhost']  # Already boolean
lis['total_reviews'] = lis['number_of_reviews']
lis['avg_rating'] = lis['review_scores_rating']
lis['review_frequency'] = lis['reviews_per_month']

# Availability and Demand
lis['availability_365'] = lis['availability_90']  # How often is it available?
lis['occupancy_rate'] = 1 - (lis['availability_365'] / 365)  # Inverse of availability

In [79]:
# Location Features
COPENHAGEN_CENTER_LAT = 55.6761
COPENHAGEN_CENTER_LON = 12.5683

# Distance to center
lis['distance_to_center_km'] = np.sqrt(
    (lis['latitude'] - COPENHAGEN_CENTER_LAT)**2 + 
    (lis['longitude'] - COPENHAGEN_CENTER_LON)**2
) * 111  # Rough conversion to kilometers

# Neighborhood density
lis['listings_in_neighborhood'] = lis.groupby('neighbourhood_cleansed')['id'].transform('count')

In [80]:
# Price Features
# Neighborhood price comparison
lis['neighborhood_avg_price'] = lis.groupby('neighbourhood_cleansed')['price_DKK'].transform('mean')
lis['price_vs_neighborhood'] = lis['price_DKK'] / lis['neighborhood_avg_price']

# Room type price comparison
lis['room_type_avg_price'] = lis.groupby('room_type')['price_DKK'].transform('mean')
lis['price_vs_room_type'] = lis['price_DKK'] / lis['room_type_avg_price']

# Value indicators
lis['price_per_person'] = lis['price_DKK'] / lis['accommodates']
lis['price_per_bedroom'] = lis['price_DKK'].div(lis['bedrooms'].where(lis['bedrooms'] > 0, 1))

In [81]:
# Host Features
lis['host_experience_years'] = (pd.Timestamp.now() - lis['host_since']).dt.total_seconds() / (365.25 * 24 * 60 * 60)
lis['host_listings_ratio'] = lis['host_total_listings_count'] / lis['listings_in_neighborhood']

# Response Quality
lis['host_response_speed'] = pd.Categorical(lis['host_response_time'], 
    categories=['within an hour', 'within a few hours', 'within a day', 'a few days or more'], 
    ordered=True).codes

In [82]:
# Review Features
lis['days_since_last_review'] = (pd.Timestamp.now() - lis['last_review']).dt.total_seconds() / (24 * 60 * 60)
lis['review_rate'] = lis['number_of_reviews'] / lis['host_experience_years']

# Calculate review score variance
review_score_cols = [col for col in lis.columns if col.startswith('review_scores_')]
lis['review_score_variance'] = lis[review_score_cols].var(axis=1)

In [83]:
# Calendar Features
cal['date'] = pd.to_datetime(cal['date'])
cal['is_weekend'] = cal['date'].dt.dayofweek >= 5
cal['is_holiday'] = cal['date'].dt.month.isin([6, 7, 8, 12])  # Summer and December

# Aggregate to listing level
calendar_features = cal.groupby('listing_id').agg({
    'price_USD': ['mean', 'std'],
    'is_weekend': 'mean',  # Proportion of weekend days
    'is_holiday': 'mean',  # Proportion of holiday days
    'available': 'mean'    # Proportion of available days
}).reset_index()

In [84]:
# Print summary of new features
print("\nNew Feature Summary:")
new_features = lis.columns.difference(original_columns)
for col in new_features:
    print(f"\n{col}:")
    print(lis[col].describe())

# Check for any issues in new features
print("\nChecking for issues in new features:")
print(lis[new_features].isnull().sum())


New Feature Summary:

avg_rating:
count    20909.000000
mean         4.087925
std          1.760318
min          0.000000
25%          4.590000
50%          4.860000
75%          5.000000
max          5.000000
Name: avg_rating, dtype: float64

days_since_last_review:
count    20909.000000
mean      1854.225337
std       3153.659065
min        227.626360
25%        247.626360
50%        435.626360
75%        927.626360
max       9173.626360
Name: days_since_last_review, dtype: float64

distance_to_center_km:
count    20909.000000
mean         3.733427
std          2.014880
min          0.096576
25%          2.259250
50%          3.445322
75%          4.762564
max         13.204881
Name: distance_to_center_km, dtype: float64

host_experience_years:
count    20909.000000
mean         8.230373
std          3.243080
min          0.625945
25%          6.487683
50%          8.872351
75%         10.605411
max         25.116020
Name: host_experience_years, dtype: float64

host_listings_ratio:


### 9. Final Status Check

In [85]:
print("Listings shape:", lis.shape)
print("Calendar shape:", cal.shape) 
print("Reviews shape:", rev.shape)

Listings shape: (20909, 90)
Calendar shape: (7631731, 8)
Reviews shape: (366636, 5)


### 10. Save Processed Datasets

In [86]:
lis.to_parquet('data/processed/03_listings_cleaned.parquet')
cal.to_parquet('data/processed/03_calendar_cleaned.parquet') 
rev.to_parquet('data/processed/03_reviews_cleaned.parquet')